In [5]:
from PIL import Image
import struct
import os

def amiga_planar_to_png(gfx_file, palette, output_dir):
    """
    Convert Amiga planar tiles to PNG files
    
    gfx_file: path to 'data/gfx' file
    palette: list of 16 RGB tuples [(R,G,B), ...]
    output_dir: where to save PNG files
    """
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    with open(gfx_file, 'rb') as f:
        gfx_data = f.read()
    
    # 256 tiles, 64 bytes each
    for tile_id in range(256):
        offset = tile_id * 64
        tile_data = gfx_data[offset:offset+64]
        
        # Create 16×8 image
        img = Image.new('RGB', (16, 8))
        pixels = img.load()
        
        # Decode planar data
        for row in range(8):
            # Each row = 4 bitplanes × 2 bytes
            row_offset = row * 8  # 8 bytes per row
            
            bp0 = struct.unpack('>H', tile_data[row_offset:row_offset+2])[0]
            bp1 = struct.unpack('>H', tile_data[row_offset+2:row_offset+4])[0]
            bp2 = struct.unpack('>H', tile_data[row_offset+4:row_offset+6])[0]
            bp3 = struct.unpack('>H', tile_data[row_offset+6:row_offset+8])[0]
            
            # Extract 16 pixels from bitplanes
            for x in range(16):
                bit = 15 - x  # MSB first
                
                # Combine bits from 4 bitplanes
                color_index = ((bp0 >> bit) & 1) | \
                             (((bp1 >> bit) & 1) << 1) | \
                             (((bp2 >> bit) & 1) << 2) | \
                             (((bp3 >> bit) & 1) << 3)
                
                pixels[x, row] = palette[color_index]
        
        # Save tile with hex filename (e.g., 00.png, 0A.png, FF.png)
        img.save(f'{output_dir}/tile_{tile_id:02X}.png')
        
    print(f"Exported {256} tiles to {output_dir}/")

# Amiga palette from the game (COLS in the ASM)
# NOTE: The game DOUBLES these values before writing to hardware!
# See SETCOL routine: add.l d0,d0
amiga_colors = [
    0x000,  # 0: Black
    0x444,  # 1: Gray
    0x777,  # 2: Light gray (actually white when doubled)
    0x330,  # 3: Dark yellow
    0x760,  # 4: Orange/Red-orange
    0x020,  # 5: Dark green
    0x040,  # 6: Green
    0x232,  # 7: Gray-green
    0x077,  # 8: Cyan
    0x110,  # 9: Very dark yellow/brown
    0x743,  # 10: Brown/tan
    0x773,  # 11: Yellow
    0x017,  # 12: Blue
    0x730,  # 13: Red-orange
    0x300,  # 14: Dark red
    0x710,  # 15: Red
]

# Convert Amiga 0-15 values to RGB 0-255
# The game doubles the color values, so we do too
palette = []
for color in amiga_colors:
    # Double the color value first (as the game does)
    color = color * 2
    
    r = (color >> 8) & 0xF   # Red channel
    g = (color >> 4) & 0xF   # Green channel
    b = color & 0xF          # Blue channel
    
    # Replicate nibble (0x7 -> 0x77, 0xF -> 0xFF)
    r = (r << 4) | r
    g = (g << 4) | g
    b = (b << 4) | b
    
    palette.append((r, g, b))

# Print palette for verification
print("Palette colors:")
for i, (r, g, b) in enumerate(palette):
    print(f"  Color {i:2d}: RGB({r:3d}, {g:3d}, {b:3d}) - #{r:02X}{g:02X}{b:02X}")

# Run the conversion
amiga_planar_to_png('gfx', palette, 'tiles_output')



Palette colors:
  Color  0: RGB(  0,   0,   0) - #000000
  Color  1: RGB(136, 136, 136) - #888888
  Color  2: RGB(238, 238, 238) - #EEEEEE
  Color  3: RGB(102, 102,   0) - #666600
  Color  4: RGB(238, 204,   0) - #EECC00
  Color  5: RGB(  0,  68,   0) - #004400
  Color  6: RGB(  0, 136,   0) - #008800
  Color  7: RGB( 68, 102,  68) - #446644
  Color  8: RGB(  0, 238, 238) - #00EEEE
  Color  9: RGB( 34,  34,   0) - #222200
  Color 10: RGB(238, 136, 102) - #EE8866
  Color 11: RGB(238, 238, 102) - #EEEE66
  Color 12: RGB(  0,  34, 238) - #0022EE
  Color 13: RGB(238, 102,   0) - #EE6600
  Color 14: RGB(102,   0,   0) - #660000
  Color 15: RGB(238,  34,   0) - #EE2200
Exported 256 tiles to tiles_output/
